# Verifying false-positive rates by simulation

This notebook contains simulations to show that the permutation-based approach to group-sequential testing used in this package effectively controls the false-positive rate, even when analyzing the data sequentially (i.e. stopping data collection when the pattern of interest is significant, and continuing otherwise). 

In these simulations, we'll use functions from the `niseq.max_test` module, which perform permutation _t_-tests with a $t$-max correction for multiple comparisons, to illustrate how `niseq` can control the familywise error rate across multiple looks during data collection. While we don't show simulations for our cluster-based tests here (those take a lot longer to run), note that the same backend functions, `niseq._permutation.generate_permutation_dist` and `niseq._permutation.find_thresholds`, are used under the hood of every statistical test implemented in `niseq`, so the permutation scheme scrutinized here is the exact same as that used in our cluster-level tests.

Note that these simulations will only give us approximate results. Feel free to increase `N_SIMULATIONS` to get something closer to the true false-positive rate.

In [1]:
from mne.parallel import parallel_func
from niseq.max_test import (
    sequential_permutation_t_test_1samp, 
    sequential_permutation_test_indep
)
import numpy as np

N_SIMULATIONS = 10000

On each simulation below, we'll generate null data and pretend we look at it five times throughout the intended course of data collection, and we'll compare the false positive rates attained when we reject the null hypothesis whenever $p \leq 0.05$ and $p \leq \alpha_\text{adjusted}$ at at least one look time.

In [2]:
def one_simulation(seed, 
                n_tests = 1,
                tail = 0, 
                indep = False,
                look_times = np.linspace(100, 500, 5).astype(int)):
    
    # generate null data 
    rng = np.random.default_rng(seed)
    x = rng.normal(loc = 0, size = (look_times[-1], n_tests))
    
    # run sequential test
    if indep:
        conds = rng.choice([0, 1], look_times[-1])
        _, p, adj_alpha, _ = sequential_permutation_test_indep(
            x, conds, look_times, n_max = look_times[-1], 
            tail = tail,
            seed = seed,
            verbose = False
        ) 
    else:
        _, p, adj_alpha, _ = sequential_permutation_t_test_1samp(
            x, look_times, n_max = look_times[-1], 
            tail = tail,
            seed = seed,
            verbose = False
        ) 
        
    # reject if p-val crosses sig threshold at any look time
    return np.array([np.any(p < .05), np.any(p < adj_alpha)]) 


def run_simulations(n_simulations, n_tests = 1, tail = 0, indep = False, n_jobs = -1):
    parallel, p_func, _ = parallel_func(one_simulation, n_jobs)
    out = parallel(p_func(seed, n_tests, tail, indep) for seed in range(n_simulations))
    rejections = np.stack(out)
    fpr = rejections.mean(0)
    print('False positive rate without correction: ' + str(fpr[0]))
    print('False positive rate *with* correction: ' + str(fpr[1]))

## One Sample Test

In [3]:
# permutation t-test
run_simulations(N_SIMULATIONS, indep = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1442
False positive rate *with* correction: 0.047


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 45.1min finished


In [4]:
# t-max correction for 100 tests
run_simulations(N_SIMULATIONS, n_tests = 100, indep = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 32.4min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1784
False positive rate *with* correction: 0.0439


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 92.0min finished


## Independent Sample Test

In [5]:
# permutation t-test
run_simulations(N_SIMULATIONS, indep = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1402
False positive rate *with* correction: 0.0465


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 72.5min finished


In [6]:
# t-max correction
run_simulations(N_SIMULATIONS, n_tests = 100, indep = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 642 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 1794 tasks      | elapsed: 24.5min
[Parallel(n_jobs=-1)]: Done 2172 tasks      | elapsed: 29.7min
[Parallel(n_jobs=-1)]: Done 2586 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 3036 tasks      | elapsed: 41.5min
[Parallel(n_jobs=-1)]: Done 3522 tasks      | elapsed: 48.1min
[Parallel(n_jobs=-1)]: Done 4044 tasks      | ela

False positive rate without correction: 0.1715
False positive rate *with* correction: 0.0458


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 136.5min finished
